# 項目14～16

### 必要なモジュール

In [1]:
import random
random.seed(1234)

import logging
from pprint import pprint
from sys import stdout as STDOUT

# Write all output to a temporary directory
import atexit
import gc
import io
import os
import tempfile

TEST_DIR = tempfile.TemporaryDirectory()
atexit.register(TEST_DIR.cleanup)

# Make sure Windows processes exit cleanly
OLD_CWD = os.getcwd()
atexit.register(lambda: os.chdir(OLD_CWD))
os.chdir(TEST_DIR.name)

def close_open_files():
    everything = gc.get_objects()
    for obj in everything:
        if isinstance(obj, io.IOBase):
            obj.close()

atexit.register(close_open_files)

<function __main__.close_open_files()>

## 項目14 key引数を使い複雑な基準でソートする

`list`には様々な基準に基づいて`list`の要素を順序に従って整列するsortメソッドがある。

デフォルトのsortでは、listの内容を要素の自然な順序の昇順に並べる。

例えば、次のコードは整数を小さいものから大きいものに並べる。

In [2]:
# Example 1
numbers = [93, 86, 11, 68, 70]
numbers.sort()
print(numbers)

[11, 68, 70, 86, 93]


sortメソッドは自然な順序のあるほとんどの組み込み型（文字列、浮動小数点数など）で動作する。

ここでオブジェクトに対してどのような動作が起きるか見てみよう。

例えば、建築現場で使う様々な工具を表すクラスを、インスタンスを出力できるよう`__repr__`メソッド（項目75参照）を含めて定義する。

In [3]:
# Example 2
class Tool:
    def __init__(self, name, weight):
        self.name = name
        self.weight = weight

    def __repr__(self):
        return f'Tool({self.name!r}, {self.weight})'

tools = [
    Tool('level', 3.5),
    Tool('hammer', 1.25),
    Tool('screwdriver', 0.5),
    Tool('chisel', 0.25),
]

この型のオブジェクトのソートを行おうとすると、（クラスで定義されていない）比較のための特殊メソッドを`sort`メソッドが呼び出そうとするため失敗する。

In [4]:
# Example 3
try:
    tools.sort()
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "C:\Users\su10_\AppData\Local\Temp\ipykernel_29048\3010603424.py", line 3, in <cell line: 2>
    tools.sort()
TypeError: '<' not supported between instances of 'Tool' and 'Tool'


クラスに（整数のような）自然な順序がない場合には、必要な特殊メソッド（項目73参照）を定義して、追加のパラメータがなくても`sort`が動作するようにできる。

しかし、より一般的な場合には、オブジェクトで複数の順序付けをサポートする必要があり、自然な順序の定義だけでは意味がない。

しばしば、オブジェクトの属性値によってソートする場合がある。このユースケースをサポートするために、`sort`メソッドには関数を引数とする`key`パラメータがある。

`key`関数には単一引数としてソートされるリストの要素が渡される。戻り値は、ソートのために使用する比較可能な（自然な順序を持つ）値でなければならない。

次のコードでは、`Tool`オブジェクトのリストを名前の英字淳にソートできる`key`パラメータのための関数をlambdaキーワードを用いて定義する。

In [5]:
# Example 4
print('Unsorted:', repr(tools))
tools.sort(key=lambda x: x.name)
print('\nSorted:  ', tools)

Unsorted: [Tool('level', 3.5), Tool('hammer', 1.25), Tool('screwdriver', 0.5), Tool('chisel', 0.25)]

Sorted:   [Tool('chisel', 0.25), Tool('hammer', 1.25), Tool('level', 3.5), Tool('screwdriver', 0.5)]


同様に重さでソートするラムダ関数も定義でき、`sort`メソッドに`key`パラメータとして渡すことができる。

In [6]:
# Example 5
tools.sort(key=lambda x: x.weight)
print('By weight:', tools)

By weight: [Tool('chisel', 0.25), Tool('screwdriver', 0.5), Tool('hammer', 1.25), Tool('level', 3.5)]


`key`パラメータとして渡されるラムダ関数の中で要素の属性に、（シーケンス、タプル、辞書では）要素のインデックス、あるいは、他の式を用いてアクセスすることができる。

文字列のような基本型では、`key`関数を使ってソートの前に値の変換ができる。

例えば次のコードでは、通常のソートでは大文字が小文字の前に来るところを、
大文字小文字を無視した英字順なるよう、`list`の中の要素の名前に`lower`メソッドを適用している。

In [7]:
# Example 6
places = ['home', 'work', 'New York', 'Paris']
places.sort()
print('Case sensitive:  ', places)
places.sort(key=lambda x: x.lower())
print('Case insensitive:', places)

Case sensitive:   ['New York', 'Paris', 'home', 'work']
Case insensitive: ['home', 'New York', 'Paris', 'work']


ソートに複数の基準を使う必要が生じることもある。例えば、工具のリストがあり、まず重さ、次に名前でソートしたいとする。

In [8]:
# Example 7
power_tools = [
    Tool('drill', 4),
    Tool('circular saw', 5),
    Tool('jackhammer', 40),
    Tool('sander', 4),
]

Pythonでの最も単純な方法は`tupple`型を用いることである。`tupple`は任意のPython値の変更不能なシーケンスである。

`tupple`はデフォルトで比較可能で自然な順序があり、`sort`メソッドが必要とする`__lt__`のような特殊メソッドを備えている。

`tupple`同士では`tupple`の各位置の要素について実装されている特殊メソッドで順に比較していく。

ある工具が別の工具よりも重い場合にこれがどうなるかを次に示す。

In [9]:
# Example 8
saw = (5, 'circular saw')
jackhammer = (40, 'jackhammer')
assert not (jackhammer < saw)  # 各要素で比較が行われている

タプルの最初の位置の要素（上例では重さ）が等しい場合、タプルの比較は次の位置の要素の比較に移る。

In [10]:
# Example 9
drill = (4, 'drill')
sander = (4, 'sander')
assert drill[0] == sander[0]  # Same weight
assert drill[1] < sander[1]   # Alphabetically less
assert drill < sander         # Thus, drill comes first

この`tupple`の比較方式を用いて、工具をまずその重さでソートし、等しい場合には名前でソートすることができる。

次のコードでは、優先順位順にソートする二つの属性の特性を返す`key`関数を定義している。

In [11]:
# Example 10
power_tools.sort(key=lambda x: (x.weight, x.name))
print(power_tools)

[Tool('drill', 4), Tool('sander', 4), Tool('circular saw', 5), Tool('jackhammer', 40)]


`key`関数で`tuple`を返す場合の制限は、すべての基準に対してソートの方向が同じ（全部が昇順か降順）ことである。

`sort`メソッドに`reverse`パラメータを渡すと、`tuple`の両方の基準に同じように影響する（`sander`の方が`drill`より前に来ることに注意）。

In [12]:
# Example 11
power_tools.sort(key=lambda x: (x.weight, x.name),
                 reverse=True)  # Makes all criteria descending
print(power_tools)

[Tool('jackhammer', 40), Tool('circular saw', 5), Tool('sander', 4), Tool('drill', 4)]


数値については、マイナス単項演算子を用い`key`関数でソートの昇順/降順を混在させることができる。

マイナス演算子は、他には影響せずに、返された`tuple`の値の一つを符号変換することで、`sort`の順番を逆転させることができる。

次のコードでは`weight`を降順、`name`を昇順にソートしている。

In [13]:
# Example 12
power_tools.sort(key=lambda x: (-x.weight, x.name))
print(power_tools)

[Tool('jackhammer', 40), Tool('circular saw', 5), Tool('drill', 4), Tool('sander', 4)]


ただし、単項マイナス演算子はすべての型に使える訳ではない。

In [14]:
# Example 13
try:
    power_tools.sort(key=lambda x: (x.weight, -x.name),
                     reverse=True)
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "C:\Users\su10_\AppData\Local\Temp\ipykernel_29048\2522468187.py", line 3, in <cell line: 2>
    power_tools.sort(key=lambda x: (x.weight, -x.name),
  File "C:\Users\su10_\AppData\Local\Temp\ipykernel_29048\2522468187.py", line 3, in <lambda>
    power_tools.sort(key=lambda x: (x.weight, -x.name),
TypeError: bad operand type for unary -: 'str'


このように、`name`の順序を逆転させるため、Pythonには安定ソートアルゴリズムが用意されている。

`list`型の`sort`メソッドは、`key`関数が互いに等しいという値を返した時には、入力リストでの順番を保持する。

これは、同じリストに`sort`を複数回呼び出し、異なる基準を組み合わせられるということを意味している。

次のコードでは上で行ったように、`weight`が降順、`name`が昇順となる`sort`を別々に二回呼び出して行っている。

In [15]:
# Example 14
power_tools.sort(key=lambda x: x.name)   # Name ascending

power_tools.sort(key=lambda x: x.weight, # Weight descending
                 reverse=True)

print(power_tools)

[Tool('jackhammer', 40), Tool('circular saw', 5), Tool('drill', 4), Tool('sander', 4)]


上のコードの挙動を確認しよう。最初の`sort`では名前を英字順にしている。

In [16]:
# Example 15
power_tools.sort(key=lambda x: x.name)
print(power_tools)

[Tool('circular saw', 5), Tool('drill', 4), Tool('jackhammer', 40), Tool('sander', 4)]


二番目の`sort`が`weight`の降順で呼ばれると、'sander'と'drill'が同じ重さの4であることに気づく。

これにより、`sort`メソッドは、この2要素をもとの`list`に現れていたのと同じ順序になるようにソートするため、`name`の相対順序は昇順が保たれる。

In [17]:
# Example 16
power_tools.sort(key=lambda x: x.weight,
                 reverse=True)
print(power_tools)

[Tool('jackhammer', 40), Tool('circular saw', 5), Tool('drill', 4), Tool('sander', 4)]


これと同じ方式がさまざまなソート基準を好きな方向に組み合わせる際に使うことができる。

参集の`list`で保持するものを逆のシーケンスでソートするように確認すればよいだけである。

この例では、`weight`を降順に、`name`を昇順にしたいため、`name`をソートしてから、次に`weight`をソートする必要があった。

先ほどの例での「`key`関数で`tuple`を返し、マイナス単項演算子を使って異なるソート順を混在させる」手法の方が読みやすく、必要なコード量も少なく済む。

この例のように複数回ソートを呼び出すのは必要な場合にのみするべきである。

### 覚えておくこと

- `list`型の`sort`メソッドは、リストの内容を文字列、整数、タプルなどの自然な順序に並べ替えるのに使う
- `sort`メソッドは、特殊メソッドを使って順序付けするメソッドを定義しないとオブジェクトで動作しないが、一般的にはあまりない
- `sort`メソッドの`key`パラメータを使い、`list`の各要素をソートする値を返すヘルパー関数を与えることができる
- `key`関数で`tuple`を返すことにより、複数のソート基準を組み合わせることができる。単項マイナス演算子が、その型で許されるソート順を逆転するのに使える
- マイナス演算子を使えない型では、`sort`メソッドをさまざまな`key`関数と`reverse`値に対して、最も低いランクの`sort`呼び出しから最も高いランクの`sort`呼び出しまで順に複数回呼び出すことで組み合わせたソートが出来る。

## 項目15 dictの挿入順序に依存する場合は注意する

Python3.5以前では、`dict`をイテレートすると任意の順序でキーが返されるようになっていた。

要素を挿入した順序とは無関係な順序になってしまっていた。

例えば、動物とその子どもの呼び名の辞書を作って出力した例を示す（項目75参照）。

In [18]:
# Example 1
# Python 3.5
baby_names = {
    'cat': 'kitten',
    'dog': 'puppy',
}
print(baby_names)

{'cat': 'kitten', 'dog': 'puppy'}


※3.5だとこの左右が逆

辞書を作成したときの順序は、'cat'、'dog'でしたが出力では逆になってしまう。

これは、それまでの辞書型の実装が、組み込み型関数`hash`とPython起動時に与えられる乱数のシードとのハッシュ表アルゴリズムに依存してたことが原因である。

この振る舞いは挿入順序が辞書でのキーの順序とは合致せず、プログラム実行ごとにシャッフルされる原因になっていました。

Python3.6以降はPythonの仕様として、辞書は挿入順を保持するようになった。

In [19]:
# Example 2
baby_names = {
    'cat': 'kitten',
    'dog': 'puppy',
}
print(baby_names)

{'cat': 'kitten', 'dog': 'puppy'}


Python3.5以前では、イテレーション順序に依存する`dict`の全メソッドは、`keys`,`values`,`items`,`popitem`を含めて、同じようにランダムに見える振る舞いであった。

In [20]:
# Example 3
# Python 3.5
baby_names = {
    'cat': 'kitten',
    'dog': 'puppy',
}
print(list(baby_names.keys()))
print(list(baby_names.values()))
print(list(baby_names.items()))
print(baby_names.popitem())  # Randomly chooses an item

['cat', 'dog']
['kitten', 'puppy']
[('cat', 'kitten'), ('dog', 'puppy')]
('dog', 'puppy')


これらのメソッドも、一貫した挿入順序となった。

In [36]:
baby_names = {
    'cat': 'kitten',
    'dog': 'puppy',
}

# Example 4
print(list(baby_names.keys()))
print(list(baby_names.values()))
print(list(baby_names.items()))
print(baby_names.popitem())  # Last item inserted

['cat', 'dog']
['kitten', 'puppy']
[('cat', 'kitten'), ('dog', 'puppy')]
('dog', 'puppy')


この変更は`dict`型とその実装に依存していた他のPythonの機能に多くの影響を与えた。

`**kwargs`パラメータ（「項目23」参照）を含めて、関数のキーワード引数も従来はランダムな順序だったため関数呼び出しのデバッグが難しくなっていた。

In [22]:
# Example 5
# Python 3.5
def my_func(**kwargs):
    for key, value in kwargs.items():
        print('%s = %s' % (key, value))

my_func(goose='gosling', kangaroo='joey')

goose = gosling
kangaroo = joey


今ではキーワード引数の順序もプログラマが関数を呼び出した元の順序になるよう保持されている。

In [23]:
# Example 6
def my_func(**kwargs):
    for key, value in kwargs.items():
        print(f'{key} = {value}')

my_func(goose='gosling', kangaroo='joey')

goose = gosling
kangaroo = joey


クラスもインスタンス辞書に`dict`型を使っている。Pythonの古いバージョンでは、オブジェクトのフィールドの順序はランダムだった。

In [24]:
# Example 7
# Python 3.5
class MyClass:
    def __init__(self):
        self.alligator = 'hatchling'
        self.elephant = 'calf'

a = MyClass()
for key, value in a.__dict__.items():
    print('%s = %s' % (key, value))


alligator = hatchling
elephant = calf


これも今ではインスタンスフィールドの代入順序が`__dict__`に反映されている。

In [25]:
# Example 8
class MyClass:
    def __init__(self):
        self.alligator = 'hatchling'
        self.elephant = 'calf'

a = MyClass()
for key, value in a.__dict__.items():
    print(f'{key} = {value}')

alligator = hatchling
elephant = calf


    長らく組み込みモジュール`collections`には挿入順序を保持する`OrderedDict`クラスがあった。

    （Python3.7以降）このクラスの振る舞いは標準`dict`型と同じだが、`OrderedDict`の性能・特性はまったく異なる。

    キーの挿入と`popitem`呼び出しが高頻度（例えばLRUキャッシュの実装）なら標準`dict`型よりも`OrderedDict`の方が良い（「項目70」参照）

辞書が挿入順序を保持する方式は、Python言語仕様の一部となっている。上のような言語機能についてはこの振る舞いに頼ることができ、設計するクラスや関数のAPIの一部とすることができる。

しかし、挿入順序の振る舞いが、辞書を扱う場合に常に保持されると仮定すべきではない。

Pythonではプログラマが`list`、`dict`などに合致する標準プロトコルをエミュレーションするコンテナ型を定義することが簡単にできる（項目43参照）。

Pythonは静的型付けではなく、ほとんどのコードは厳格なクラス階層に基づく代わりに、オブジェクトの振る舞いがデファクト型に基づく*ダックタイピング*（動的型付けに対応したオブジェクト指向プログラミング言語に特徴的な、型付けのスタイルのひとつ）に依存している。

これは予測しない結果を生じる原因になりうる。

例えば、一番かわいい動物の赤ちゃんというコンテストの結果を示すプログラムを考える。まず、それぞれの投票数を数えた辞書を考える。

In [26]:
# Example 9
votes = {
    'otter': 1281,
    'polar bear': 587,
    'fox': 863,
}

この投票データを処理して、動物の順位を空の辞書に登録する関数を定義する。辞書はUI要素のデータモデルである。

In [27]:
# Example 10
def populate_ranks(votes, ranks):
    names = list(votes.keys())
    names.sort(key=votes.get, reverse=True)
    for i, name in enumerate(names, 1):
        ranks[name] = i

どの動物がコンテストで優勝したかを知らせる関数も定義しよう。

次の関数は、上の`populate_ranks`が辞書`ranks`に昇順で内容を登録しており、先頭のキーが優勝だと仮定する。

In [28]:
# Example 11
def get_winner(ranks):
    return next(iter(ranks))

これらの関数が設計した通りに動作して、結果が期待通りになっているかを確認する。

In [29]:
# Example 12
ranks = {}
populate_ranks(votes, ranks)
print(ranks)
winner = get_winner(ranks)
print(winner)

{'otter': 1, 'fox': 2, 'polar bear': 3}
otter


さて、このプログラムの要求が変わったとする。結果を示すUI要素を順位ではなく、英字順に表示することになったとする。

そのために、組み込みモジュール`collection.abc`を使って、新たな辞書的クラス`SortedDict`を作り、英字順に内容をイテレートするとしよう。

In [30]:
# Example 13
from collections.abc import MutableMapping

class SortedDict(MutableMapping):
    def __init__(self):
        self.data = {}

    def __getitem__(self, key):
        return self.data[key]

    def __setitem__(self, key, value):
        self.data[key] = value

    def __delitem__(self, key):
        del self.data[key]

    # keyでソートするように定義
    def __iter__(self):
        keys = list(self.data.keys())
        keys.sort()
        for key in keys:
            yield key

    def __len__(self):
        return len(self.data)

# SortedDictの挙動確認
my_dict = SortedDict()
my_dict['otter'] = 1
my_dict['cheeta'] = 2
my_dict['anteater'] = 3
my_dict['deer'] = 4

assert my_dict['otter'] == 1

assert 'cheeta' in my_dict
del my_dict['cheeta']
assert 'cheeta' not in my_dict

expected = [('anteater', 3), ('deer', 4), ('otter', 1)]
assert list(my_dict.items()) == expected

assert not isinstance(my_dict, dict)

前に定義した関数の標準`dict`の代わりに、この`SortedDict`インスタンスを使うが標準辞書のプロトコルに適合しているためエラーは生じない。

しかし、実行結果は正しくないものが出力される。

In [31]:
# Example 14
sorted_ranks = SortedDict()
populate_ranks(votes, sorted_ranks)
print(sorted_ranks.data)
winner = get_winner(sorted_ranks)
print(winner)

{'otter': 1, 'fox': 2, 'polar bear': 3}
fox


これは上で実装した`get_winner`が、辞書のイテレーションで挿入順序が`populate_ranks`に一致していると仮定しているため、この問題が起こってしまう。

（`dict`ではなく`SortedDict`を使っているため、この仮定は成り立たない。）

よって優勝者として返された値は、英字順で先頭の「'fox'」となっている。

この問題を解決するには3つの方法がある。

一つ目は`get_winner`関数を再実装し、`ranks`辞書が特定のイテレーション順になっていると仮定しないということであり、最も保守的で頑健な解法である。

In [32]:
# Example 15
def get_winner(ranks):
    for name, rank in ranks.items():
        if rank == 1:
            return name

winner = get_winner(sorted_ranks)
print(winner)

otter


二つ目の方法は、関数の先頭で`ranks`の型が期待通りかチェックして、もしそうでないなら例外を出す方法である。

これは一つ目の方法よりも実行性能が優れている。

In [33]:
# Example 16
try:
    def get_winner(ranks):
        if not isinstance(ranks, dict): # 1番目の引数に指定したオブジェクトが2番目の引数に指定したデータ型と等しいかどうかを返す
            raise TypeError('must provide a dict instance')
        return next(iter(ranks))
    
    assert get_winner(ranks) == 'otter'
    
    get_winner(sorted_ranks)
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "C:\Users\su10_\AppData\Local\Temp\ipykernel_29048\3818594816.py", line 10, in <cell line: 2>
    get_winner(sorted_ranks)
  File "C:\Users\su10_\AppData\Local\Temp\ipykernel_29048\3818594816.py", line 5, in get_winner
    raise TypeError('must provide a dict instance')
TypeError: must provide a dict instance


三つ目の方法は、型ヒントを用いて`get_winner`に渡される値が`dict`インスタンスで辞書的振る舞いをする`MutableMapping`でないことを確認する方法である（「項目90」参照）。

次のコードでは、上のコードにアノテーションを付け、mypyツールを`strict`モードで実行したものである。

In [34]:
# Example 17
# Check types in this file with: python -m mypy <path>

from typing import Dict, MutableMapping

def populate_ranks(votes: Dict[str, int],
                   ranks: Dict[str, int]) -> None:
    names = list(votes.keys())
    names.sort(key=votes.get, reverse=True)
    for i, name in enumerate(names, 1):
        ranks[name] = i

def get_winner(ranks: Dict[str, int]) -> str:
    return next(iter(ranks))

from typing import Iterator, MutableMapping

class SortedDict(MutableMapping[str, int]):
    def __init__(self) -> None:
        self.data: Dict[str, int] = {}

    def __getitem__(self, key: str) -> int:
        return self.data[key]

    def __setitem__(self, key: str, value: int) -> None:
        self.data[key] = value

    def __delitem__(self, key: str) -> None:
        del self.data[key]

    def __iter__(self) -> Iterator[str]:
        keys = list(self.data.keys())
        keys.sort()
        for key in keys:
            yield key

    def __len__(self) -> int:
        return len(self.data)

votes = {
    'otter': 1281,
    'polar bear': 587,
    'fox': 863,
}

sorted_ranks = SortedDict()
populate_ranks(votes, sorted_ranks)
print(sorted_ranks.data)
winner = get_winner(sorted_ranks)
print(winner)


{'otter': 1, 'fox': 2, 'polar bear': 3}
fox


これは正しく`dict`と`MutableMapping`型との間の不整合を検出して、不正な使い方をエラーだと通知する。

これは静的型安全性と実行性能の最良の組み合わせと言える。

### 覚えておくこと

- Python3.7以降では、`dict`インスタンスの内容をイテレーションすると、キーが最初に挿入された順番が保持される
- Pythonでは、`dict`インスタンスではないが辞書のようにふるまうオブジェクトを簡単に定義できる。そのような型では、挿入順序が保持されるとは仮定できない。
- 辞書的クラスで注意するには3つの方法がある。
  - 挿入順序に依存しないコードを書く
  - 実行時に`dict`型か明示的にチェックする
  - 型ヒントと静的解析を使って`dict`値の要件をチェックする